In [1]:
import pandas as pd

In [2]:
df_meta = pd.read_csv('data/dhsipums_macro_meta.csv')
df_data = pd.read_csv('data/dhsipums_macro_data.csv')

In [3]:
def dropna(df, thresh):
    """
    Drop columns that has less than threshold samples
    and then drop samples that has missing values
    """
    return df.dropna(thresh=thresh, axis=1).dropna()

In [4]:
# Get the list of shrinked data frames
shrinked = [dropna(df_data, thresh) for thresh in range(df_data.shape[0])]
# Only keep the ones that has the minimum data in it
shrinked = [df for df in shrinked if df.shape[0] > 1 and df.shape[1] > 3]

In [5]:
deduped = {}
for df in shrinked:
    cols = df.columns.tolist()
    cols.sort()
    cols_key = ''.join(cols)
    deduped[cols_key] = df

In [6]:
import synapseclient
from synapseclient import File
syn = synapseclient.Synapse()
syn.login()

Welcome, Eric Yi Wu!



In [7]:
folder = syn.get('syn6135634')
for df in deduped.values():
    fout = 'data/dhsipums_data_reduced_' + str(df.shape[0]) + '_' + str(df.shape[1]) + '.csv'
    df.to_csv(fout, index=False)
    syn.store(File(fout, parent=folder))


##################################################
 Uploading file to Synapse storage 
##################################################
Uploading [####################]100.00%     7.4kB/7.4kB dhsipums_data_reduced_90_17.csv Done...
    
##################################################
 Uploading file to Synapse storage 
##################################################
Uploading [####################]100.00%     3.5kB/3.5kB dhsipums_data_reduced_136_6.csv Done...
    
##################################################
 Uploading file to Synapse storage 
##################################################
Uploading [####################]100.00%     7.0kB/7.0kB dhsipums_data_reduced_105_14.csv Done...
    
##################################################
 Uploading file to Synapse storage 
##################################################
Uploading [####################]100.00%     5.3kB/5.3kB dhsipums_data_reduced_131_9.csv Done...
    
##########################################

In [12]:
# Which of the reduced data set has the stunt features
stunts = ['Anthro_BMI_F_Mean', 'Anthro_Stunt_Child_Pct', 'Anthro_StuntSev_Child_Pct']
for dfkey in deduped.keys():
    for col in deduped[dfkey].columns:
        if col == 'Anthro_BMI_F_Mean' or col == 'Anthro_Stunt_Child_Pct' or col == 'Anthro_StuntSev_Child_Pct':
            print(deduped[dfkey].shape)

(75, 19)
(50, 20)
(39, 21)
